In [1]:
!pip install numpy pandas matplotlib seaborn scikit-learn --quiet
!pip install scikit-learn --updgrade --quiet
# !pip install opendatasets --quiet
!pip install plotly --quiet
# import opendatasets as od
import pandas as pd
import numpy as np
# dataset_url = 'https://www.kaggle.com/competitions/house-prices-advanced-regression-techniques'
# od.download(dataset_url)


Usage:   
  pip install [options] <requirement specifier> [package-index-options] ...
  pip install [options] -r <requirements file> [package-index-options] ...
  pip install [options] [-e] <vcs project url> ...
  pip install [options] [-e] <local project path> ...
  pip install [options] <archive url/path> ...

no such option: --updgrade


Shape of data


In [2]:
data_dir = './data'
data_df = pd.read_csv(f'{data_dir}/train.csv', nrows=100000)
data_df.shape

(1460, 81)

In [3]:
import plotly.express as px

# compute correlations with SalePrice
corr_series = data_df.corr(numeric_only=True)['SalePrice'].drop('SalePrice')
corr_df = corr_series.reset_index().rename(columns={'index':'feature', 'SalePrice':'corr'})

# sort by absolute correlation so strongest dependencies appear first
corr_df = corr_df.sort_values('corr', key=lambda x: x.abs(), ascending=False)

# bar chart (signed correlation, colored by sign/magnitude)
fig = px.bar(
    corr_df,
    x='feature',
    y='corr',
    color='corr',
    color_continuous_scale='RdBu',
    title='Correlation of features with SalePrice',
    labels={'corr':'corr_with_SalePrice', 'feature':'feature'}
)
fig.update_layout(yaxis=dict(range=[-1,1]), xaxis_title='Feature', yaxis_title='Correlation')
fig.show(renderer="browser")

//Remove Field are BsmtFinType2 , BsmtHalfBath,MiscVal,Id,LowQualFinSF,YrSold,MoSold,3SsnPorch

In [4]:
remove_cols = ['BsmtFinType2', 'BsmtHalfBath', 'MiscVal', 'Id', 'LowQualFinSF', 'YrSold', 'MoSold', '3SsnPorch']




In [5]:
data_df = data_df.drop(remove_cols, axis=1)

In [6]:
# Show number of nulls per column
def checkNullValues(df):
    null_counts = df.isnull().sum().sort_values(ascending=False)
    print(null_counts[null_counts > 0])


MasVnrType = None 
MasVnrArea,GarageYrBlt , LotFrontage = Numerical
Electrical  drop it 


In [7]:
def clean_missing_values(df):
    # Drop 'Electrical'
    if 'Electrical' in df.columns:
        df.drop('Electrical', axis=1, inplace=True)
    
    # Fill MasVnrType with 'None'
    if 'MasVnrType' in df.columns:
        df['MasVnrType'].fillna('None', inplace=True)
    
    # Columns to exclude from filling (numerical columns)
    exclude_cols = ['MasVnrArea', 'GarageYrBlt', 'LotFrontage', 'MasVnrType']
    
    # Fill other categorical nulls with 'NA'
    for col in df.select_dtypes(include='object').columns:
        if col not in exclude_cols:
            df[col].fillna('NA', inplace=True)
    
    return df




In [8]:
data_df = clean_missing_values(data_df)

/tmp/ipykernel_64829/2073745228.py:8: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.



/tmp/ipykernel_64829/2073745228.py:16: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)'

In [9]:
def most_Val(df, col_name):
    """
    Returns the most common (mode) value of a numerical column.
    """
    if col_name in df.columns:
        # Drop nulls first
        mode_value = df[col_name].mode(dropna=True)
        if not mode_value.empty:
            return mode_value[0]
        else:
            return None
    else:
        return None

# Usage for GarageYrBlt
common_year = most_Val(data_df, 'GarageYrBlt')
print("Most common Garage Year Built:", common_year)
if common_year is not None:
    data_df['GarageYrBlt'].fillna(common_year, inplace=True)


Most common Garage Year Built: 2005.0


/tmp/ipykernel_64829/1016363599.py:19: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [10]:
checkNullValues(data_df)


LotFrontage    259
MasVnrArea       8
dtype: int64


In [11]:
area = most_Val(data_df, 'MasVnrArea')
area
data_df['MasVnrArea'].fillna(area, inplace=True)

/tmp/ipykernel_64829/3485532086.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





In [12]:
area = most_Val(data_df, 'LotFrontage')
area
data_df['LotFrontage'].fillna(area, inplace=True)

/tmp/ipykernel_64829/2632795815.py:3: FutureWarning:

A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.





input & catogrical cols

In [13]:
def Input_Categorical_Cols(df):
    input_col = [ col for col in df.columns if col != 'SalePrice' ]
    output_col = 'SalePrice'

    numerical_cols = ['LotFrontage','LotArea','YearBuilt','YearRemodAdd','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','TotalBsmtSF','1stFlrSF','2ndFlrSF','LowQualFinSF','GrLivArea','BsmtFullBath','BsmtHalfBath','FullBath','HalfBath','Bedroom','Kitchen','TotRmsAbvGrd','Fireplaces','GarageYrBlt','GarageCars','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch','3SsnPorch','ScreenPorch','PoolArea','MiscVal',]
    numerical_cols = [col for col in numerical_cols if col not in remove_cols]
    categorical_cols = [col for col in input_col if col not in numerical_cols]
    return input_col, output_col, numerical_cols, categorical_cols

In [14]:
input_col, output_col, numerical_cols, categorical_cols = Input_Categorical_Cols(data_df);